In [2]:
# =======================
# Cell 1: Imports & Config
# =======================
import os
import time
import random
import warnings
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score,
    f1_score, precision_score, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d  # ✅ NEW: Open3D for point cloud loading

# Import custom model & configs
import sys
sys.path.append("../Utils")
from models import DGCNN  # You will have models.py with DGCNN implemented
import configs

# Paths
TRAIN_DIR = Path(configs.TRAIN_DIR)
TEST_DIR = Path(configs.TEST_DIR)
MODEL_DIR = Path(configs.MODEL_DIR)


# Force CPU device (no CUDA for now)
class CFG:
    num_points = 1024
    batch_size = 16
    epochs = 20
    lr = 1e-3
    device = torch.device("cpu")  # ✅ FORCE CPU
    k = 20
    emb_dims = 1024
    dropout = 0.5
    num_classes = len(os.listdir(TRAIN_DIR))

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print(f"⚠️ Running on {CFG.device} (CUDA {'available' if torch.cuda.is_available() else 'not available'})")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
⚠️ Running on cpu (CUDA not available)


In [ ]:
# ==========================
# Cell 2: Robust Point Cloud Loader
# ==========================
import chardet
import open3d as o3d
import numpy as np

def read_point_file(path):
    """
    Universal point cloud reader.
    Reads any .txt/.xyz/.pts file regardless of encoding.
    Returns: numpy array (N,3) or (N,6) if RGB is present.
    """
    with open(path, "rb") as f:
        raw = f.read()
        enc = chardet.detect(raw)["encoding"] or "latin1"
        text = raw.decode(enc, errors="ignore")

    data = []
    for line in text.strip().splitlines():
        parts = line.strip().split()
        if len(parts) < 3:
            continue
        try:
            x, y, z = map(float, parts[:3])
            if len(parts) >= 6:
                r, g, b = map(float, parts[3:6])
                data.append([x, y, z, r / 255.0, g / 255.0, b / 255.0])
            else:
                data.append([x, y, z])
        except ValueError:
            continue

    points = np.array(data, dtype=np.float32)
    return points


def farthest_point_sampling(points, n_samples=1024):
    """
    Farthest Point Sampling (FPS) using Open3D.
    Works for XYZ or XYZ+RGB.
    """
    xyz = points[:, :3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd = pcd.farthest_point_down_sample(n_samples)
    sampled_xyz = np.asarray(pcd.points, dtype=np.float32)

    # Preserve RGB if present
    if points.shape[1] > 3:
        sampled_rgb = []
        for sp in sampled_xyz:
            idx = np.linalg.norm(xyz - sp, axis=1).argmin()
            sampled_rgb.append(points[idx, 3:6])
        sampled_rgb = np.array(sampled_rgb, dtype=np.float32)
        return np.hstack((sampled_xyz, sampled_rgb))

    return sampled_xyz


ModuleNotFoundError: No module named 'chardet'

In [13]:
# =======================
# Cell 3: Custom Dataset
# =======================
class PointCloudDataset(Dataset):
    def __init__(self, root_dir, n_points=1024, transform=None):
        self.root_dir = Path(root_dir)
        self.classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.files = []
        self.labels = []
        self.n_points = n_points
        self.transform = transform

        # Collect file paths
        for idx, cls in enumerate(self.classes):
            for f in (self.root_dir / cls).glob("*.*"):
                self.files.append(f)
                self.labels.append(idx)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        label = self.labels[idx]

        points = read_point_file(file)
        # Downsample using FPS to n_points
        if len(points) > self.n_points:
            points = farthest_point_sampling(points, n_samples=self.n_points)
        else:
            # Pad if not enough points
            pad_size = self.n_points - len(points)
            points = np.vstack([points, np.zeros((pad_size, points.shape[1]), dtype=np.float32)])

        if self.transform:
            points = self.transform(points)

        return torch.tensor(points.T, dtype=torch.float32), label  # (C, N)


In [14]:
# =======================
# Cell 4: Dataloaders
# =======================
train_dataset = PointCloudDataset(TRAIN_DIR, n_points=CFG.num_points)
test_dataset = PointCloudDataset(TEST_DIR, n_points=CFG.num_points)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=0)

print(f"✅ Loaded {len(train_dataset)} training samples, {len(test_dataset)} test samples")
print(f"Classes: {train_dataset.classes}")


✅ Loaded 557 training samples, 134 test samples
Classes: ['Ash', 'Beech', 'Douglas Fir', 'Oak', 'Pine', 'Red Oak', 'Spruce']


In [16]:
# =======================
# Cell 5: Model Setup
# =======================
args = type('', (), {})()  # empty object for args
args.k = CFG.k
args.emb_dims = CFG.emb_dims
args.dropout = CFG.dropout

model = DGCNN(args, output_channels=CFG.num_classes).to(CFG.device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=CFG.lr)

print(f"✅ DGCNN model initialized with {CFG.num_classes} classes")


✅ DGCNN model initialized with 7 classes


In [17]:
# =======================
# Cell 6: Training Loop
# =======================
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct = 0, 0
    for points, labels in loader:
        points, labels = points.to(CFG.device), torch.tensor(labels).to(CFG.device)

        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

    return total_loss / len(loader), correct / len(loader.dataset)


def evaluate(model, loader):
    model.eval()
    preds_list, labels_list = [], []
    with torch.no_grad():
        for points, labels in loader:
            points = points.to(CFG.device)
            outputs = model(points)
            preds_list.extend(outputs.argmax(dim=1).cpu().numpy())
            labels_list.extend(labels)

    acc = accuracy_score(labels_list, preds_list)
    return acc, preds_list, labels_list


best_acc = 0
for epoch in range(CFG.epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    test_acc, _, _ = evaluate(model, test_loader)

    print(f"Epoch {epoch+1}/{CFG.epochs} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), MODEL_DIR / "best_dgcnn.pth")
        print(f"💾 Model saved with acc {best_acc:.4f}")


[Open3D WARNING] Read PTS failed with exception: vector too long
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Spruce\65_88.txt (format: auto).
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Douglas Fir\61_45.txt (format: auto).
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Douglas Fir\61_20.txt (format: auto).
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Spruce\35_23.txt (format: auto).
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Douglas Fir\31_59.txt (format: auto).
[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for C:\Users\lenovo\Desktop\TDDP\dataset\train\Douglas Fir\31_54.txt (format

ValueError: ❌ Error reading C:\Users\lenovo\Desktop\TDDP\dataset\train\Red Oak\Baum7_256 brb_pssu╠êd- Cloud.txt: 'utf-8' codec can't decode byte 0xa6 in position 147: invalid start byte

In [ ]:
# =======================
# Cell 7: Evaluation
# =======================
best_model = DGCNN(args, output_channels=CFG.num_classes).to(CFG.device)
best_model.load_state_dict(torch.load(MODEL_DIR / "best_dgcnn.pth"))

acc, preds, labels = evaluate(best_model, test_loader)
print(f"✅ Final Test Accuracy: {acc:.4f}")

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=train_dataset.classes, yticklabels=train_dataset.classes, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
